In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install captum
!pip install transformers==3

1.13.0+cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 30.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.6/754.6 KB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 63.1 MB/s eta 0:00:00
  Preparing metadata (setup.p

In [2]:
from google.colab import files
uploaded = files.upload()

Saving gcn_2l_model.py to gcn_2l_model.py


In [3]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import matplotlib.pyplot as plt
import torch
from torch_geometric.datasets import TUDataset
import numpy as np
from termcolor import colored
from torchsummary import summary
from torch.autograd import Variable
from keras import backend as K
from statistics import mean
from sklearn import metrics
from copy import deepcopy
from captum.attr import Saliency
from scipy.spatial.distance import hamming
from itertools import zip_longest
from time import perf_counter
import csv
import torch.nn as nn
import torch_geometric.nn as gnn
from torch import Tensor
from torch_geometric.typing import OptPairTensor, Adj, OptTensor, Size
from typing import Callable, Union, Tuple
from torch_sparse import SparseTensor
from time import perf_counter
import random
import pandas
from torch_geometric.loader import DataLoader
import sklearn
import gcn_2l_model

1.13.0+cu116


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
class Model_Train_Test(object):
  def __init__(self, task, start_epoch):
    self.BATCH_SIZE = 64
    self.size_of_hidden_layers = 7
    self.Explainability_name = 'Model Train Test'
    self.Task_name = 'Graph Classification'
    self.checkpoint_directory_Classifier = "/content/drive/My Drive/Explainability Methods/" + str(self.Explainability_name) + " on " + str(self.Task_name) + "/Model/model_classifier.pt"



    self.df = pandas.read_csv("/content/drive/My Drive/Explainability Methods/Train and Test Indexes on Graph Classification/Experimental Results/train_test_indexes.csv")

    self.read_training_list_indexes__ = self.df['Train Indexes']
    self.read_test_list_indexes__ = self.df['Test Indexes']

    self.read_training_list_indexes, self.read_test_list_indexes = self.prepare_indexes_of_train_test_datasets()
    #print(self.read_training_list_indexes)
    #print(self.read_test_list_indexes)

    self.train_dataset, self.test_dataset, self.feature_size, self.number_classes = self.prepare_train_test_datasets()
    #print(len(self.train_dataset))

    self.train_loader, self.test_loader = self.prepare_train_test_batches()

    self.GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=self.feature_size, dim_hidden=self.feature_size, dim_output=self.number_classes)
    self.GCN_model.eval()
    self.criterion = torch.nn.CrossEntropyLoss()

    self.optimizer = self.my_optimizer()
    self.Visualization_Parameter = 50
    self.Model_Saving_Parameter = 100

    self.GCN_model_training_time_per_epoch = []
    self.GCN_model_training_Acc_per_epoch = []
    
    if task == 'Train':
      self.run_model_training(start_epoch)
    elif task == 'Test':
      acc, preds = self.model_test(start_epoch)
      print("Accuracy: ", acc, "Predicted Labels: ", preds)
    




  def prepare_indexes_of_train_test_datasets(self):
    read_test_list_indexes__ = self.read_test_list_indexes__.dropna()
    read_training_list_indexes__ = self.read_training_list_indexes__.dropna()
    read_test_list_indexes = []
    read_training_list_indexes = []
    for element in read_training_list_indexes__:
      read_training_list_indexes.append(int(element))
    for element in read_test_list_indexes__:
      read_test_list_indexes.append(int(element))
    return read_training_list_indexes, read_test_list_indexes

  def prepare_train_test_datasets(self):

    dataset = TUDataset(root='data/TUDataset', name='MUTAG')
    torch.manual_seed(123456)
    dataset = dataset.shuffle()

    train_dataset = []
    test_dataset = []
    for index in self.read_training_list_indexes:
      train_dataset.append(dataset[index])
    for index in self.read_test_list_indexes:
      test_dataset.append(dataset[index])

    feat_size = train_dataset[0].x.shape[1]


    return train_dataset, test_dataset, feat_size, dataset.num_classes

  def prepare_train_test_batches(self):
    train_loader = DataLoader(self.train_dataset, batch_size=self.BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(self.test_dataset, batch_size=self.BATCH_SIZE, shuffle=False)
    return train_loader, test_loader

  def loss_calculations(self, preds, gtruth):
    loss_per_epoch = self.criterion(preds, gtruth)
    return loss_per_epoch


  def loading_model(self, load_index):
    GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=self.feature_size, dim_hidden=self.feature_size, dim_output=self.number_classes)
    optimizer = torch.optim.Adam(params = GCN_model.parameters(), lr=1e-4)
    checkpoint = torch.load("/content/drive/My Drive/Explainability Methods/"+str(self.Explainability_name)+" on " + str(self.Task_name) + "/Model/" + str(self.Explainability_name) + " on " + str(self.Task_name) + " classifier model_" + str(load_index)+".pt")
    GCN_model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    return GCN_model, optimizer, epoch
  

  def visualize_losses(self, g_losses, epoch_history):
    g_losses_list = torch.stack(g_losses).cpu().detach().numpy()

    fig = plt.figure(figsize=(27,20))

    ax = plt.subplot2grid((3, 1), (0, 0), colspan=1)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title("GCN Model Loss in Epoch: " + str(epoch_history))

    ax.plot(g_losses_list, color='r')

    plt.savefig('/content/drive/My Drive/Explainability Methods/'+str(self.Explainability_name)+' on ' + str(self.Task_name) + '/Experimental Results/'+'Loss_til_epoch_{:04d}.png'.format(epoch_history))
    plt.show()  
  
  def my_optimizer(self):
    return torch.optim.RMSprop(self.GCN_model.parameters(), lr=0.001)


  def train_step(self):
    GCN_model_loss_batch = []
    GCN_model_Pred_Labels = []
    Real_Labels = []
    
    self.GCN_model.train()
    self.optimizer.zero_grad()
    for batched_data in self.train_loader:
      GCN_model_out = self.GCN_model(batched_data.x, batched_data.edge_index, batched_data.batch)
      pred = GCN_model_out.argmax(dim=1)
      GCN_model_Pred_Labels.extend(pred.detach().tolist())
      Real_Labels.extend(batched_data.y.detach().tolist())
      #print(pred.detach().tolist())
      #correct += int((pred == data.y).sum())  # Check against ground-truth labels.
      #correct / len(loader.dataset)

      GCN_model_loss = self.loss_calculations(GCN_model_out, batched_data.y)
      GCN_model_loss_batch.append(GCN_model_loss)
      GCN_model_loss.backward()
      self.optimizer.step()

    self.optimizer.step()
    # print(Real_Labels)
    # print(GCN_model_Pred_Labels)

    return torch.mean(torch.tensor(GCN_model_loss_batch)), metrics.accuracy_score(Real_Labels, GCN_model_Pred_Labels)
  


  def train(self, EPOCHS, load_index):
    GCN_model_training_loss_per_epoch = []

    for epoch in range(EPOCHS):

      start_generation = perf_counter()
      GCN_model_training_loss, training_acc = self.train_step()
      self.GCN_model_training_time_per_epoch.append(perf_counter() - start_generation)
      self.GCN_model_training_Acc_per_epoch.append(training_acc)

      print(f'Epoch: {epoch+1:03d}, {self.Explainability_name} Model Loss: {GCN_model_training_loss:.4f} Accuracy: {training_acc}')

      GCN_model_training_loss_per_epoch.append(GCN_model_training_loss)
      #break

      if (epoch + load_index + 1) % self.Visualization_Parameter == 0 and epoch > 0:
        self.visualize_losses(GCN_model_training_loss_per_epoch, epoch + load_index + 1)
      if (epoch + load_index + 1) % self.Model_Saving_Parameter == 0 and epoch > 0:
        torch.save({'epoch': epoch+load_index+1, 'model_state_dict': self.GCN_model.state_dict(), 'optimizer_state_dict': self.optimizer.state_dict(), 'loss': GCN_model_training_loss_per_epoch,}, "/content/drive/My Drive/Explainability Methods/" + str(self.Explainability_name) + " on " + str(self.Task_name) + "/Model/" + str(self.Explainability_name) + " on " + str(self.Task_name) + " classifier model_" + str(epoch + load_index + 1)+".pt")
  
  
  def run_model_training(self, start_epoch):
    EPOCHS = 200
    load_index = start_epoch
    if load_index != 0:
      model_GCN, optimizer, load_index = self.loading_model(load_index)
    self.train(EPOCHS, load_index)
  
  def model_test(self, start_epoch):
    load_index = start_epoch
    if load_index != 0:
      model_GCN, optimizer, load_index = self.loading_model(load_index)
    else:
      print("At first train the model!!!")
    predictions = []
    self.GCN_model.eval()
    correct = 0
    for batched_data in self.test_loader:  
      GCN_model_test_out = self.GCN_model(batched_data.x, batched_data.edge_index, batched_data.batch)  
      GCN_model_test_pred = GCN_model_test_out.argmax(dim=1)
      predictions.append(GCN_model_test_pred)  
      correct += int((GCN_model_test_pred == batched_data.y).sum())  
    return correct / len(self.test_loader.dataset), predictions[0].detach().tolist()




In [12]:

model_trained = Model_Train_Test('Test', 200) # Task(train or test), Start Epoch
print(model_trained)
GCN_model, optimizer, epoch = model_trained.loading_model(200)
print(GCN_model)

Accuracy:  0.7631578947368421 Predicted Labels:  [0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0]
GCN_2Layer_Model(
  (gconv1): GCNConv(7, 7)
  (gconv2): GCNConv(7, 7)
  (readout): GlobalMeanPool()
  (ffn): Linear(in_features=7, out_features=2, bias=False)
  (dropout): Dropout(p=0.5, inplace=False)
)
